In [ ]:
import os, glob, cv2, math, csv, tqdm, random, keras
import numpy as np
from PIL import Image
import util 
from keras.utils.training_utils import multi_gpu_model
from keras import backend as k
from keras import applications, optimizers, losses, layers
from keras.utils.training_utils import multi_gpu_model
from keras import layers ,losses
from keras.layers import Dropout, Input, Flatten, Dense, GlobalAveragePooling2D, Activation, LSTM, SeparableConv2D, TimeDistributed, GRU
from keras.models import Sequential, Model, load_model
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers.convolutional import Conv2D
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
def data_list_load(seq, root):
    img_path_total = []
    lab_path_total = []
    for seq_ch in seq:
        img_dir = os.path.join(root, 'sequences', seq_ch,'image_2')
        seq_pose = seq_ch + '.txt'
        lab_dir = os.path.join(root, 'poses', seq_pose)
        lab_list = util.load_dof6_labels(lab_dir)
        lab_dir = os.path.join(root, 'poses', seq_pose)
        paths = sorted(glob.glob(img_dir+'/*.png'))

        for i in tqdm.tqdm(paths):
            img_time = [[paths[i], paths[i+1], paths[i+2], 
                         paths[i+3], paths[i+4], paths[i+5], 
                         paths[i+6], paths[i+7], paths[i+8], 
                         paths[i+9], paths[i+10]] 
                        for i in range(len(paths)-10)]
        img_path_total = img_path_total + img_time
        lab_path_total = lab_path_total + lab_list[9:]
    print(len(img_path_total), len(lab_path_total))
    return img_path_total, lab_path_total

In [ ]:
def data_generator(total_img, total_lab, batch):
    total = []        
    while 1:
        idx = list(range(0,len(total_img)))
        idx_batch = random.sample(idx,batch)
        
        bat_img = []
        bat_lab = []
        
        for i in idx_batch:
            tmp_img = []
            for j in range(10):
                img = cv2.resize(np.concatenate((Image.open(total_img[i][j]),Image.open(total_img[i][j+1])),axis = 2),(512,384))/255.
                tmp_img.append(img)
            lab = total_lab[i]
            bat_img.append(np.array(tmp_img))
            bat_lab.append(lab) 
        yield np.array(bat_img), [np.array(bat_lab)[:,:3],np.array(bat_lab)[:,3:]]

In [ ]:
def test_generator(total_img, total_lab, batch):
    total = []
    idx_batch = 0        
    while 1:
        bat_img = []
        bat_lab = []
        tmp_img = []       
        for j in range(10):
            img = cv2.resize(np.concatenate((Image.open(total_img[idx_batch][j]),Image.open(total_img[idx_batch][j+1])),axis = 2),(512,384))/255.
            tmp_img.append(img)
        lab = total_lab[idx_batch]
        bat_img.append(np.array(tmp_img))
        bat_lab.append(lab) 
        idx_batch = idx_batch + 1
        if idx_batch == len(total_img)-10:
            idx_batch = 0
        yield np.array(bat_img), [np.array(bat_lab)[:,:3],np.array(bat_lab)[:,3:]]

In [ ]:
batch = 1
# train_seq = ['00','01','02','03','04','05','06','07','08']
train_seq = ['01']
train_root= '/data1/Kitti/odometry/'
train_list, train_lab = data_list_load(train_seq, train_root)
train = data_generator(train_list, train_lab, batch)

val_seq = ['09']
val_root= '/data1/Kitti/odometry/'
val_list, val_lab = data_list_load(val_seq, val_root)
val = data_generator(val_list, val_lab, batch)

In [ ]:
model_gen = load_model('../Opticalflow/20190527_mobflow_00000001.h5')
model_top = Model(inputs = model_gen.input, 
                  outputs=model_gen.layers[18].output)
model_top.summary()

In [ ]:
inputs = Input(shape=(None, 384, 512,6))
x = TimeDistributed(model_top)(inputs)
x = TimeDistributed(GlobalAveragePooling2D())(x)

x = LSTM(1024, return_sequences=True)(x)
x = LSTM(1024)(x)
rotation = Dense(512, activation='relu')(x)
rotation = Dense(3, name = 'rotation')(rotation)
position = Dense(512, activation='relu')(x)
position = Dense(3, name = 'position')(position)

In [ ]:
model = Model(inputs = inputs, outputs = [rotation, position])
#for i in range(2):
#    model.layers[i].trainable = False
model.compile(optimizer='adam', loss=['mse','mse'],loss_weights=[100,1])
model.summary()

In [ ]:
mc = keras.callbacks.ModelCheckpoint('save_new/20190528_deepvo_seqtrain_step256{epoch:08d}.h5', period=1)

In [ ]:
hist = model.fit_generator(train, steps_per_epoch=200000, epochs=10, callbacks=[mc],
                           validation_data= val, validation_steps= 5)    

In [ ]:
test_seq = ['01']
test_root= '/data1/Kitti/odometry/'
test_list, test_lab = data_list_load(test_seq, test_root)
test = test_generator(test_list, test_lab, batch)

In [ ]:
predic = []
labels = []
for i in tqdm.tqdm(range(1000)):
    test_img, test_lab = next(test)
    labels.append(np.reshape(np.concatenate((np.array(test_lab)[0],np.array(test_lab)[1]),axis=1),(1,6))[-1,:])
    predic_tmp = model.predict(test_img)
    predic.append(np.reshape(np.concatenate((np.array(predic_tmp)[0],np.array(predic_tmp)[1]),axis=1),(1,6))[-1,:])

In [ ]:
predic_np = np.array(predic)
labels_np = np.array(labels)

In [ ]:
path = util.dof6_to_eval_path(predic_np)
path_gt = util.dof6_to_eval_path(labels_np)

plt.plot(path[0,:],path[2,:],'r')
plt.plot(path_gt[0,:],path_gt[2,:],'g')
plt.show()